# adaBoost Model

#### to do:

Add VIX and VIX return in the correlation_filter 

manage names in the features: spy_volume, spy_return, spy_close, etc

In [99]:
# Import appropriate modules

import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder, MinMaxScaler
from sklearn.metrics import classification_report

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

import datetime
import numpy as np
import yfinance as yf
from datetime import datetime
from pandas.tseries.offsets import DateOffset
import hvplot
import hvplot.pandas
import matplotlib.pyplot as plt
from vix_functions import garch_fit_and_predict, correlation_filter, retrieve_yahoo_close, retrieve_yahoo_volume 

# Generation of the Features Matrix X

In [100]:
# Ticker List: VIX must be in first position
ticker_list= ["^VIX", "spy", 'XLF', 'XLE',
              'EURUSD=X', 'GBPUSD=X', 'AUDUSD=X', 'BRLUSD=X', "DX-Y.NYB","USDJPY=X", 
              '^TNX', 'ZB=F', 'ZF=F', 'PYMAX', 'NQ=F','NKD=F',
              'VWEHX','LQD', '^IRX',
              'AAPL', 'AMZN', 'GE','MU','MSFT', 'BMY', 'FDX', 'GS','PLD','NVDA',   
              "tlt", "ief", 
              "FXI", "EZU", "EEM", "EFA", 'FEZ', "^GDAXI", '^FTSE','^HSI','TPY=F','^FCHI',
              "gld", "slv", "CL=F", '261220.KS']
    
#
#    
# Energy Select Sector SPDR Fund (XLE)
# Financial Select Sector SPDR Fund (XLF)
# CAC 40 (^FCHI)
# Yen Denominated TOPIX Futures,D (TPY=F)
# HANG SENG INDEX (^HSI)
# FTSE 100 (^FTSE)
# DAX PERFORMANCE-INDEX (^GDAXI)
# SPDR EURO STOXX 50 ETF (FEZ)
# Nikkei/USD Futures,Dec-2021 (NKD=F)
# Nasdaq 100 Dec 21 (NQ=F)
#  NVIDIA Corporation (NVDA)
# '261220.KS':'WTI CRUDE FUTURE'
# 'EURUSD': Euro spot
# Treasury Yield 10 Years (^TNX) -- 1985
# U.S. Treasury Bond Futures,Dec- (ZB=F) - 2000
# Five-Year US Treasury Note Futu (ZF=F) - 2000
# American Funds U.S. Government Securities Fund Class C (UGSCX) - 2001
# PIMCO High Yield Municipal Bond Fund Class A (PYMAX)
# Vanguard High-Yield Corporate Fund Investor Shares (VWEHX) -- 1985
# iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD)
# 13 Week Treasury Bill (^IRX)
# Micron Technology, Inc. (MU)
# Microsoft Corporation (MSFT)
# Bristol-Myers Squibb Company (BMY)
# FEDEX CORP (FDX)
# The Goldman Sachs Group, Inc. (GS)
# Prologis, Inc. (PLD)


In [101]:
#

In [102]:
#


In [121]:
# Inclusion of security levels X1
def retrieve_close(close_prices_dict, ticker_list):
    for ticker in ticker_list:
        close_price = retrieve_yahoo_close(ticker, start_date='2006-07-02', end_date='2021-10-01')
        close_prices_dict[ticker] = close_price
    return close_prices_dict

close_prices_dict = {}
close_prices_dict = retrieve_close(close_prices_dict, ticker_list)
print("Completed retrieve of close prices")

close_prices_df= pd.DataFrame(close_prices_dict)

close_prices_component_df = correlation_filter(close_prices_df, min_corr=0.20, key_column='^VIX', eliminate_first_column=True)

X1=close_prices_component_df
vix=close_prices_df['^VIX']
vix_ret=close_prices_df['^VIX'].pct_change()
VIX=pd.DataFrame([vix, vix_ret]).T
VIX.columns=['VIX','VIX_ret']

X1=pd.concat([VIX,close_prices_component_df], axis=1)

Processing Close ^VIX
Processing Close spy
Processing Close EURUSD=X
Processing Close GBPUSD=X
Processing Close AUDUSD=X
Processing Close BRLUSD=X
Processing Close DX-Y.NYB
Processing Close USDJPY=X
Processing Close ^TNX
Processing Close ZB=F
Processing Close ZF=F
Processing Close PYMAX
Processing Close VWEHX
Processing Close LQD
Processing Close ^IRX
Processing Close AAPL
Processing Close AMZN
Processing Close GE
Processing Close MU
Processing Close MSFT
Processing Close BMY
Processing Close FDX
Processing Close GS
Processing Close PLD
Processing Close NVDA
Processing Close tlt
Processing Close ief
Processing Close FXI
Processing Close EZU
Processing Close EEM
Processing Close EFA
Processing Close gld
Processing Close slv
Processing Close CL=F
Processing Close 261220.KS
Completed retrieve of close prices


In [104]:
# Inclusion of security returns X2
# Include returns that are correlated more than 0.20 with the Vix return

security_returns_df= close_prices_df.pct_change()
security_returns_component_df = correlation_filter(
                                        security_returns_df, 
                                        min_corr=0.20, 
                                        key_column='^VIX', 
                                        eliminate_first_column=True 
)

X2=security_returns_component_df


#### OBSERVATION: It would be good to include VIX level and Return and check on both in case correlations changes in the future. As of now with some variables I check with VIX level, and others I check VIX return

In [105]:
# inclusion of security volume X3
volume_list = ticker_list[1:len(ticker_list)]

def retrieve_volume(volume_dict, volume_list):
    for ticker in volume_list:        
        volume = retrieve_yahoo_volume(ticker)
        volume_dict[ticker] = volume
    return volume_dict

volume_dict = {}
volume_dict = retrieve_volume(volume_dict, volume_list)
print("Completed retrieve of volume")

volume_df= pd.DataFrame(volume_dict)
volume_df_with_vix=pd.concat([vix, volume_df], axis=1)
#print(volume_df_with_vix.corr())

volume_component_df = correlation_filter(volume_df_with_vix, min_corr=0.20, key_column='^VIX', eliminate_first_column=True )
X3=volume_component_df

Processing Volume spy
Processing Volume EURUSD=X
Processing Volume GBPUSD=X
Processing Volume AUDUSD=X
Processing Volume BRLUSD=X
Processing Volume DX-Y.NYB
Processing Volume USDJPY=X
Processing Volume ^TNX
Processing Volume ZB=F
Processing Volume ZF=F
Processing Volume PYMAX
Processing Volume VWEHX
Processing Volume LQD
Processing Volume ^IRX
Processing Volume AAPL
Processing Volume AMZN
Processing Volume GE
Processing Volume MU
Processing Volume MSFT
Processing Volume BMY
Processing Volume FDX
Processing Volume GS
Processing Volume PLD
Processing Volume NVDA
Processing Volume tlt
Processing Volume ief
Processing Volume FXI
Processing Volume EZU
Processing Volume EEM
Processing Volume EFA
Processing Volume gld
Processing Volume slv
Processing Volume CL=F
Processing Volume 261220.KS
Completed retrieve of volume


In [106]:
np.mean(abs(security_returns_df['LQD']))


0.0030859154770095116

In [107]:
# Inclusion of GARCH series X4
garch_series=pd.DataFrame()

not_to_include=['^VIX', '261220.KS','VWEHX']


for ticker in ticker_list:
    
        if ticker in not_to_include:
            continue
    
        if (np.mean(abs(security_returns_df[ticker])))<=0.01:
            security_returns_df[ticker]=10*security_returns_df[ticker]
        
        garch_series[ticker]=garch_fit_and_predict(security_returns_df[ticker], ticker, horizon=1, p=1, q=1, o=1)

        if (np.mean(abs(security_returns_df[ticker])))<=0.01:
            garch_series[ticker]=100*garch_series[ticker]
            
        
X4=garch_series
X4


starting series: spy
starting series: EURUSD=X
starting series: GBPUSD=X
starting series: AUDUSD=X
starting series: BRLUSD=X
starting series: DX-Y.NYB
starting series: USDJPY=X
starting series: ^TNX
starting series: ZB=F
starting series: ZF=F
starting series: PYMAX
starting series: LQD
starting series: ^IRX
starting series: AAPL
starting series: AMZN
starting series: GE
starting series: MU
starting series: MSFT
starting series: BMY
starting series: FDX
starting series: GS
starting series: PLD
starting series: NVDA
starting series: tlt
starting series: ief
starting series: FXI
starting series: EZU
starting series: EEM
starting series: EFA
starting series: gld
starting series: slv
starting series: CL=F


,spy,EURUSD=X,GBPUSD=X,AUDUSD=X,BRLUSD=X,DX-Y.NYB,USDJPY=X,^TNX,ZB=F,ZF=F,...,NVDA,tlt,ief,FXI,EZU,EEM,EFA,gld,slv,CL=F
Date,,,,,,,,,,,,,,,,,,,,,
2007-07-03,0.000981,0.000375,0.000359,0.000767,0.000875,0.000280,0.000622,0.000122,0.000547,0.000270,...,0.000198,0.000677,0.000367,0.000217,0.000130,0.000184,0.001033,0.000717,0.000118,0.000135
2007-07-04,0.000918,0.000370,0.000355,0.000751,0.000845,0.000276,0.000593,0.000120,0.000539,0.000263,...,0.000193,0.000661,0.000357,0.000209,0.000125,0.000176,0.000986,0.000704,0.000116,0.000130
2007-07-05,0.000864,0.000365,0.000353,0.000736,0.000819,0.000274,0.000569,0.000123,0.000571,0.000283,...,0.000189,0.000688,0.000370,0.000202,0.000121,0.000169,0.000967,0.000699,0.000115,0.000129
2007-07-06,0.000812,0.000363,0.000349,0.000761,0.000805,0.000271,0.000551,0.000122,0.000567,0.000279,...,0.000185,0.000678,0.000364,0.000200,0.000117,0.000164,0.000930,0.000714,0.000120,0.000130
2007-07-09,0.000764,0.000358,0.000345,0.000769,0.000776,0.000267,0.000526,0.000121,0.000563,0.000274,...,0.000186,0.000668,0.000365,0.000195,0.000112,0.000158,0.000890,0.000717,0.000118,0.000133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-27,0.000870,0.000332,0.000421,0.000549,0.000994,0.000295,0.000418,0.000273,0.000536,0.000124,...,0.000213,0.000873,0.000350,0.000193,0.000114,0.000126,0.000966,0.000855,0.000154,0.000166
2021-09-28,0.001370,0.000331,0.000416,0.000539,0.001015,0.000302,0.000407,0.000271,0.000545,0.000122,...,0.000250,0.000909,0.000352,0.000187,0.000148,0.000130,0.001301,0.000858,0.000152,0.000157
2021-09-29,0.001273,0.000328,0.000481,0.000544,0.001001,0.000320,0.000418,0.000265,0.000537,0.000121,...,0.000246,0.000883,0.000343,0.000182,0.000142,0.000130,0.001243,0.000845,0.000166,0.000152


In [108]:
# Inclusion of return squares in X5

returns_squared_df_no_vix= security_returns_df.drop(columns='^VIX')**2
returns_squared_and_vix_level_df=pd.concat([vix,returns_squared_df_no_vix], axis=1)
returns_squared_component_df = correlation_filter(returns_squared_and_vix_level_df, min_corr=0.20, key_column='^VIX', eliminate_first_column=True)

X5=returns_squared_component_df
X5

,spy,GBPUSD=X,AUDUSD=X,BRLUSD=X,DX-Y.NYB,^TNX,ZF=F,PYMAX,VWEHX,LQD,...,NVDA,tlt,ief,FXI,EZU,EEM,EFA,gld,slv,261220.KS
Date,,,,,,,,,,,,,,,,,,,,,
2007-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-07-03,0.001313,0.000016,0.001905,0.000158,0.000000,0.000000,0.000000,0.000000,0.000000,0.000004,...,0.000003,0.003716,0.000880,0.000439,0.000042,0.000063,0.002266,0.001854,0.000020,NaN
2007-07-04,0.000000,0.000015,0.000577,0.000960,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
2007-07-05,0.000110,0.000397,0.000265,0.001020,0.000340,0.000853,0.003245,0.000364,0.000003,0.004161,...,0.000114,0.012560,0.003113,0.000012,0.000027,0.000020,0.002480,0.002443,0.000029,NaN
2007-07-06,0.002763,0.000001,0.009348,0.004338,0.000096,0.000098,0.000327,0.000366,0.000003,0.000376,...,0.000023,0.001645,0.000561,0.000630,0.000118,0.000294,0.005187,0.007027,0.000298,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-27,0.000818,0.001518,0.002129,0.004325,0.000029,0.000270,0.000068,0.000403,0.000003,0.000027,...,0.000364,0.001351,0.000718,0.000207,0.000001,0.000042,0.000039,0.000433,0.000123,0.000385
2021-09-28,0.040610,0.000568,0.000397,0.009814,0.001744,0.001135,0.000058,0.000911,0.000003,0.011403,...,0.001968,0.024264,0.001454,0.000002,0.000569,0.000167,0.047554,0.009441,0.000058,0.000478
2021-09-29,0.000283,0.013532,0.002489,0.004697,0.003695,0.000021,0.000058,0.000102,0.000000,0.000005,...,0.000077,0.000301,0.000109,0.000052,0.000024,0.000087,0.000526,0.002029,0.001595,0.001202


### Inclusion of Google Trends

In [109]:
# Upload of Google Tremds -- X6
keywords=['jobless_claims_pao', 'banking', "consumer", "depression", "gdp", "inflation",
          'unemployment', 'liquidity','cci', 'vix_word'] #[ 'pandemic' has issues of double entries]
google_trends_df=pd.DataFrame()

for keyword in keywords:
    file_path=f"./Resources/Google_trends/{keyword}.csv"
    print(file_path)
    trend=pd.read_csv(Path(file_path),
                      index_col= 'Daily', 
                      parse_dates= True,
                      infer_datetime_format=True
                     )
    #print(trend)
    google_trends_df=pd.concat([google_trends_df, trend], axis=1)
    #print(google_trends_df)

google_trends_df

./Resources/Google_trends/jobless_claims_pao.csv
./Resources/Google_trends/banking.csv
./Resources/Google_trends/consumer.csv
./Resources/Google_trends/depression.csv
./Resources/Google_trends/gdp.csv
./Resources/Google_trends/inflation.csv
./Resources/Google_trends/unemployment.csv
./Resources/Google_trends/liquidity.csv
./Resources/Google_trends/cci.csv
./Resources/Google_trends/vix_word.csv


,Jobless claims: (United States),Banking: (United States),Consumer: (United States),Depression: (United States),GDP: (United States),inflation,unemployment: (United States),Liquidity: (United States),cci,VIX:(United States),f
Daily,,,,,,,,,,,
2004-01-01,NaN,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2004-01-02,NaN,33,54,81,0.0,43.0,0.0,0.0,0.0,0.0,1.0
2004-01-03,NaN,97,22,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2004-01-04,NaN,27,22,100,0.0,53.0,0.0,0.0,0.0,0.0,1.0
2004-01-05,NaN,64,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2021-10-04,NaN,78,95,48,23.0,38.0,16.0,16.0,0.0,0.0,1.0
2021-10-05,NaN,63,65,26,24.0,78.0,19.0,25.0,0.0,15.0,1.0
2021-10-06,NaN,72,87,51,23.0,32.0,12.0,66.0,51.0,29.0,1.0


In [110]:
maximum_date=vix.index.max()
maximum_date

Timestamp('2021-10-01 00:00:00', freq='B')

In [111]:
# Working on preparing Google-trends data

# Unifying google dates with VIX
minimum_date=vix.index.min()
maximum_date=vix.index.max()

google_trends_df=google_trends_df.loc[minimum_date:maximum_date,:]
#print(google_trends_df.iloc[0,:])

# Filling missing first values with 0
google_trends_df.iloc[0,:]=google_trends_df.iloc[0,:].fillna(0)
#print(google_trends_df)

vix_google_trends_df=pd.concat([vix, google_trends_df], axis=1)

#print(vix_google_trends_df.head())

vix_google_trends_df=vix_google_trends_df.interpolate(method="polynomial", order=2) #fillna(0)
vix_google_trends_df

/Users/Paola/Applications/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:1700: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


,^VIX,Jobless claims: (United States),Banking: (United States),Consumer: (United States),Depression: (United States),GDP: (United States),inflation,unemployment: (United States),Liquidity: (United States),cci,VIX:(United States),f
2007-07-02,15.400000,0.000000,88,61,0,0.0,20.0,21.0,22.0,0.0,0.0,1.0
2007-07-03,14.920000,-0.000046,76,54,0,0.0,0.0,64.0,0.0,0.0,0.0,1.0
2007-07-04,15.275253,-0.000093,77,33,0,0.0,0.0,0.0,0.0,89.0,0.0,1.0
2007-07-05,15.480000,-0.000139,66,57,0,0.0,22.0,45.0,12.0,0.0,0.0,1.0
2007-07-06,14.720000,-0.000185,69,56,0,0.0,21.0,22.0,0.0,65.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-27,18.760000,NaN,64,58,52,47.0,21.0,22.0,25.0,0.0,45.0,1.0
2021-09-28,23.250000,NaN,65,77,51,47.0,53.0,16.0,50.0,0.0,44.0,1.0
2021-09-29,22.559999,NaN,80,44,50,23.0,35.0,18.0,16.0,0.0,44.0,1.0
2021-09-30,23.139999,NaN,68,98,77,48.0,43.0,24.0,58.0,0.0,45.0,1.0


In [112]:
# Filtering by correlation X6

google_trends_component_df = correlation_filter(
                                vix_google_trends_df, 
                                min_corr=0.05, 
                                key_column='^VIX', 
                                eliminate_first_column=True)

X6=google_trends_component_df

# We will interpolate so we can fill the missing data only on Google Trends
pro_interpolation_of_X6=pd.concat([vix, X6], axis=1)
pro_interpolation_of_X6=pro_interpolation_of_X6.interpolate(method="polynomial", order=2, axis=0)
pro_interpolation_of_X6
X6 = pro_interpolation_of_X6.iloc[:,1:]
X6

,Jobless claims: (United States),Banking: (United States),Consumer: (United States),inflation,VIX:(United States)
2007-07-02,0.000000,88,61,20.0,0.0
2007-07-03,-0.000046,76,54,0.0,0.0
2007-07-04,-0.000093,77,33,0.0,0.0
2007-07-05,-0.000139,66,57,22.0,0.0
2007-07-06,-0.000185,69,56,21.0,0.0
...,...,...,...,...,...
2021-09-27,NaN,64,58,21.0,45.0
2021-09-28,NaN,65,77,53.0,44.0
2021-09-29,NaN,80,44,35.0,44.0
2021-09-30,NaN,68,98,43.0,45.0


# INCLUSION OF LAGGED SERIES

In [113]:
# Inclusion of different lags of data from the X -- up to n_lag



# Inclusion of Economic and Financial Series X7

In [114]:
#Economic Series
# Upload of csv files -- X7
keywords=['JobClaimsWeeklySeries', 'vix_put_call_ratio']
economic_series_df=pd.DataFrame()

for keyword in keywords:
    file_path=f"./Resources/Economic_and_financial_Series/{keyword}.csv"
    print(file_path)
    new_series=pd.read_csv(Path(file_path),
                      index_col= 'DATE', 
                      parse_dates= True,
                      infer_datetime_format=True
                     )
    #print(new_series)
    # Adjustment due to weekend data. We are going to assign data on the weekends to Friday, since are going to be 
    # consider for the the prediction of Monday
    economic_series_df=pd.concat([economic_series_df, new_series], axis=1)
    print(economic_series_df.tail())

economic_series_df 
economic_series_change_df = economic_series_df.pct_change().add_suffix('_change')

./Resources/Economic_and_financial_Series/JobClaimsWeeklySeries.csv
              ICSA
DATE              
2021-09-04  312000
2021-09-11  335000
2021-09-18  351000
2021-09-25  364000
2021-10-02  326000
./Resources/Economic_and_financial_Series/vix_put_call_ratio.csv
                ICSA  VIX Put/Call Ratio
DATE                                    
2021-09-04  312000.0                 NaN
2021-09-11  335000.0                 NaN
2021-09-18  351000.0                 NaN
2021-09-25  364000.0                 NaN
2021-10-02  326000.0                 NaN


In [115]:
# Working on indexes available

# Changes of columns that are on a weekend - concat with vix to add week days
vix_economic= pd.concat([vix,economic_series_df,economic_series_change_df ],axis=1)
vix_economic['ICSA'] = vix_economic['ICSA'].shift(1)
vix_economic['ICSA_change'] = vix_economic['ICSA_change'].shift(1)
vix_economic['VIX Put/Call Ratio']= vix_economic['VIX Put/Call Ratio'].fillna(0)

# Applying interpolation to appropiate columns. Levels: interpolation, changes: zeros
vix_economic.loc[:,economic_series_df.columns]=vix_economic.loc[:,economic_series_df.columns].interpolate(method="polynomial", order=2, axis=0)
vix_economic.loc[:,economic_series_change_df.columns]=vix_economic.loc[:,economic_series_change_df.columns].fillna(0)

#print(vix_economic)

#Filtering for available dates
economic_series_ready_df = vix_economic.loc[minimum_date:maximum_date,:]
economic_series_ready_df = economic_series_ready_df.iloc[:,1:]

X7=economic_series_ready_df
X7.shape

(4463, 4)

In [119]:
# SPY volatility on varios tracks
spy_volatility_30_days = security_returns_df.rolling(window=30).std()
spy_volatility_60_days = security_returns_df.rolling(window=60).std()
spy_volatility_90_days = security_returns_df.rolling(window=90).std()
spy_volatility_260_days = security_returns_df.rolling(window=260).std()
spy_volatility_360_days = security_returns_df.rolling(window=360).std()
spy_volatility_10_days = security_returns_df.rolling(window=10).std()
spy_volatility_20_days = security_returns_df.rolling(window=20).std()
spy_volatility_180_days = security_returns_df.rolling(window=180).std()
spy_volatility_200_days = security_returns_df.rolling(window=200).std()
spy_volatility_120_days = security_returns_df.rolling(window=120).std()

# GENERATION OF THE FEATURE MATRIX **X**

In [ ]:
# Concatenation of all sources of data
XY=pd.concat([X1, X2, X3, X4, X5, X6,X7], axis=1)
print(XY.shape)

XY.dropna(subset = ['VIX', 'VIX_ret'])
print(XY.shape)

# Interpolation is not applied to numerical variables. We are just going to drop those.
print(f"XY.shape: {XY.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X1.shape: {X1.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X2.shape: {X2.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X3.shape: {X3.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X4.shape: {X4.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X5.shape: {X5.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X6.shape: {X6.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X7.shape: {X7.shape}, {XY.index.min()}, {XY.index.max()} ")
XY.fillna(0) #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
# Set the Signal column
threshold=0.030
vix_ret=vix.pct_change()
XY["Signal"] = 0.0

XY.loc[(XY['VIX_ret'] >= threshold), 'Signal'] = 1

# # Generate the trading signals 1 (entry) or -1 (exit)
# # where 1 is when the ^VIX is greater than 3.6%.
# # where 0 is when the ^VIX  is less than 3.6%.
#for index, row in XY.iterrows():
#    if row["VIX_ret"] >= 0.036:
#        XY.loc[index, "Signal"] = 1.0

# Review the DataFrame
print(XY["Signal"].head())    
XY["Signal"].value_counts()
XY.loc[XY["Signal"]==1, 'VIX_ret']
#XY.shape  ## 3981

In [ ]:
vix_ret=vix.pct_change()
vix_ret[vix_ret>=threshold].index
vix_ret.shape


# How many values of the vix we missed due to missing data on other series
compare=pd.concat([XY.loc[XY["Signal"]==1, 'VIX_ret'],vix_ret[vix_ret>=threshold] ], axis=1)
missing_dates=compare.loc[compare["VIX_ret"]!=compare["^VIX"]]
missing_dates=missing_dates.index
missing_dates
vix[missing_dates]

In [ ]:
# Define the target set y using the Signal column
y = XY["Signal"]
# Display a sample of y
y

In [ ]:
# Define features set X by selecting all columns but Signal
X = XY.drop(columns=["Signal"]).shift().dropna()


# Review the features DataFrame
X.head()

In [ ]:
X.corr()

In [ ]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
training_begin = X.index.min()
training_end = X.index.min() + DateOffset(months=50)

X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

X_test = X.loc[training_end+DateOffset(days=1):]
y_test = y.loc[training_end+DateOffset(days=1):]


In [ ]:
# Create a StandardScaler instance
scaler =  StandardScaler() # MinMaxScaler() #

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [ ]:
# Instance AdaBoost
# Initiate the model instance
adaboost_model=AdaBoostClassifier(n_estimators=120)
adaboost_model


In [ ]:
# Fit the model 
adaboost_model =adaboost_model.fit(X_train_scaled, y_train)

pred_adaboost=adaboost_model.predict(X_test_scaled)

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
adaboost_report=classification_report(y_test, pred_adaboost)

# Print the classification report
print("         AdaBoost Classification Report")
print(adaboost_report)


In [ ]:
# Number of estimators? 

for n in range (50,200, 10):
    # Instance AdaBoost
    # Initiate the model instance
    adaboost_model=AdaBoostClassifier(n_estimators=n)

    # Fit the model 
    adaboost_model =adaboost_model.fit(X_train_scaled, y_train)
    pred_adaboost=adaboost_model.predict(X_test_scaled)
    print (n)
    # Use a classification report to evaluate the model using the predictions and testing data
    adaboost_report=classification_report(y_test, pred_adaboost)

    # Print the classification report
    print("         AdaBoost Classification Report")
    print(adaboost_report)
#120 highest 1-recall
#150 good overall accuracy, but lower 1-recall
